### Notebook to run RAG setup

In [1]:
import os
import json
import getpass
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

import bs4 
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, Annotated, TypedDict
from langchain_core.prompts import PromptTemplate

# from typing import List


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
vector_store = InMemoryVectorStore(embeddings)

In [6]:
# # Load and chunk contents of the blog
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

In [7]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://petguide.dk/hundefoder-maerker/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("entry-content single-page", "entry-title", "entry-meta uppercase is-xsmall")
        )
    ),
)
docs = loader.load()

In [8]:
# initiate the text splitter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
# split the documents into chunks 
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 31 sub-documents.


In [21]:
docs[0].page_content

'Hundefoder mærker – Dit overblik over de forskellige mærker!\nUdgivet den 22. juli 20201. august 2023 af Cecile A \nDer findes alverdens hundefoder mærker, og det kan være svært at vælge imellem dem alle sammen. Det er forskelligt, hvad mærkerne har fokus på, og noget er bedre til f.eks. allergiske hunde end andet. Skifter du fra et andet mærke, er det vigtigt at du gør det i løbet af en periode på 7 dage.\n Det skal foregå langsomt, så din hund kan vænne sig til det nye foder. Laver du blandinger med det gamle foder og det nye foder, burde din hund ikke reagere voldsomt på skiftet. Der er mange meninger om, hvilken hundefoder, der er bedst, men det er meget forskelligt, hvad de individuelle hunde er til. De har også forskellige smagsløg, som vi mennesker, og nogle hunde kan også være kræsne.\nNår vi taler om foder til hunde, så skelnes der mellem tørfoder og vådfoder. \n\nGodbidsforslag til din hund\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\nCrunchy Snack m. Struds & Brombær 200 g\n29

In [9]:
# add all sub-documents to the vector store
document_ids = vector_store.add_documents(documents=all_splits)

In [26]:
# collect prompt (template) from the hub
prompt = hub.pull("rlm/rag-prompt")


prompt_template = """Brug følgende stykker kontekst til at besvare spørgsmålet i slutningen. 
Hvis du ikke kender svaret, så sig bare, at du ikke ved det, og prøv ikke at opdigte et svar.
Svar med maksimalt tre sætninger og hold svaret så kortfattet men præcist som muligt.
Vær høflig i dit svar.

{context} 

Spørgsmål: {question} 

Hjælpsomt svar:"""

custom_rag_prompt = PromptTemplate.from_template(prompt_template)

In [27]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [29]:
# define retriever to retrieve documents from the vector store
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = custom_rag_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [30]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [33]:
result = graph.invoke({"question": "Hvilket fodermærke er rig på protein, fedt, er kornfrit og produceres i Canada?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='40329028-f2dd-4395-8272-2eb48b3e8674', metadata={'source': 'https://petguide.dk/hundefoder-maerker/', 'start_index': 12524}, page_content='Pronature Holistic\nPronature Holistic stammer fra Canada, som har verdens strengeste krav til dyreernæring. Derfor indeholder foderet heller ingen kunstige farvestoffer eller smagsstoffer. Det er foder, der er produceret med bæredygtighed og 100% naturlige og økologiske ingredienser. Pronature Holistic tilbyder en bred vifte af perfekt sammensatte og velbalancerede opskrifter tilpasset din hund, og desuden så er det velegnet til hunde med hud- og pelsproblemer. Ingredienserne sikrer en god fordøjelse og at foderet er velsmagende og frisk. Deres mål er, at tilbyde de ingredienser, der på bedste vis holder dit kæledyr sundt og velfungerende.'), Document(id='fdaeed85-2012-4997-8a02-a80f094e164c', metadata={'source': 'https://petguide.dk/hundefoder-maerker/', 'start_index': 11187}, page_content='Orijen\nOrijen er canadisk og de t